In [4]:
from yaml import load
from yaml import CLoader as Loader
from planktonsdm.tune import tune
from planktonsdm.functions import example_data

In [5]:
#If you want to debug the package without repackaging, you can read files locally:

#import sys
#sys.path.insert(0, '/home/phyto/planktonSDM/planktonsdm/')
#from tune import tune
#from functions import example_data

In [6]:
# Setting up the model

with open('/home/phyto/planktonSDM/configuration/2-phase_ensemble.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

X, y = example_data(y_name =  "Coccolithus pelagicus", n_samples=500, n_features=5, noise=20, 
                    random_state=model_config['seed'])


m = tune(X, y, model_config)

scale X = True


In [7]:
'''
1-phase Random forest classifier
'''
m.train(model="rf", classifier=True)

training classifier
Fitting 3 folds for each of 16 candidates, totalling 48 fits


exported model to:/home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_clf.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_clf.sav
[0.87460317 0.69270298 0.76698641]
clf balanced accuracy 0.78
execution time: 7.152557373046875e-07 seconds


In [8]:
'''
1-phase Random forest regressor
'''
m.train(model="rf", regressor=True)

training regressor
Fitting 3 folds for each of 16 candidates, totalling 48 fits
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_reg.sav
reg rRMSE: 25%
reg rMAE: 18%
reg R2: 0.78
execution time: 4.76837158203125e-07 seconds


In [9]:
'''
1-phase Random forest regressor -  testing log transformations
'''
m.train(model="rf", regressor=True, log="both")

training regressor
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 16 candidates, totalling 48 fits
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_reg.sav
reg rRMSE: 25%
reg rMAE: 18%
reg R2: 0.78
execution time: 7.152557373046875e-07 seconds


In [10]:
'''
2-phase Random forest regressor
'''
m.train(model="rf", classifier=True, regressor=True)

training classifier
Fitting 3 folds for each of 16 candidates, totalling 48 fits
exported model to:/home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_clf.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_clf.sav
[0.87460317 0.69270298 0.76698641]
clf balanced accuracy 0.78
training regressor
Fitting 3 folds for each of 16 candidates, totalling 48 fits
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_reg.sav
reg rRMSE: 25%
reg rMAE: 18%
reg R2: 0.78
training zero-inflated regressor
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Coccolithus pelagicus_zir.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Coccolithus pelagicus_zir.sav
zir rRMSE: 28%
zir rMAE: 17%
zir R2: 0.7
execution time: 4.76837158203125e-07 seconds


In [11]:
'''
1-phase XGboost classifier
'''
m.train(model="xgb", classifier=True)

training classifier
Fitting 3 folds for each of 1 candidates, totalling 3 fits
exported model to:/home/phyto/CoccoML/ModelOutput/xgb/model/Coccolithus pelagicus_clf.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/xgb/scoring/Coccolithus pelagicus_clf.sav
[0.72222222 0.61780576 0.60751399]
clf balanced accuracy 0.65
execution time: 4.76837158203125e-07 seconds


In [12]:
'''
1-phase XGboost regressor
'''
m.train(model="xgb", regressor=True)

training regressor
Fitting 3 folds for each of 1 candidates, totalling 3 fits
exported model to: /home/phyto/CoccoML/ModelOutput/xgb/model/Coccolithus pelagicus_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/xgb/scoring/Coccolithus pelagicus_reg.sav
reg rRMSE: 43%
reg rMAE: 34%
reg R2: 0.33
execution time: 9.5367431640625e-07 seconds


In [13]:
'''
1-phase KNN classifier
'''
m.train(model="knn", classifier=True)

training classifier
Fitting 3 folds for each of 1 candidates, totalling 3 fits
exported model to:/home/phyto/CoccoML/ModelOutput/knn/model/Coccolithus pelagicus_clf.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/knn/scoring/Coccolithus pelagicus_clf.sav
[0.59259259 0.53571429 0.55555556]
clf balanced accuracy 0.56
execution time: 4.76837158203125e-07 seconds


In [14]:
'''
1-phase KNN regressor
'''
m.train(model="knn", regressor=True)

training regressor
Fitting 3 folds for each of 1 candidates, totalling 3 fits
exported model to: /home/phyto/CoccoML/ModelOutput/knn/model/Coccolithus pelagicus_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/knn/scoring/Coccolithus pelagicus_reg.sav
reg rRMSE: 39%
reg rMAE: 28%
reg R2: 0.46
execution time: 7.152557373046875e-07 seconds
